In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

Importing all the required libraries for reading the HTML + Cleaning the HTML

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

Checking if the page has been read correctly

In [4]:
title = soup.title
print(title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [5]:
table = soup.find('table')

In [6]:
rows = table.find_all('td')

In [8]:
str_cells = str(rows)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()

Cleaning the data table and converting it into a DataFrame

In [9]:
list = cleantext.split("\n")
df = pd.DataFrame(list)
df1 = df[0].str.split(',', expand=True)
df1.head(10)

,0,1,2,3
0,[M1A,Not assigned,Not assigned,None
1,,M2A,Not assigned,Not assigned
2,,M3A,North York,Parkwoods
3,,M4A,North York,Victoria Village
4,,M5A,Downtown Toronto,Harbourfront
5,,M5A,Downtown Toronto,Regent Park
6,,M6A,North York,Lawrence Heights
7,,M6A,North York,Lawrence Manor
8,,M7A,Queen's Park,Not assigned
9,,M8A,Not assigned,Not assigned


In [10]:
#Naming the columns as required
df1 = df1.drop(columns=0)
df1.iloc[0,0] = 'M1A'
df1.columns=["PostalCode","Borough","Neighbourhood"]
df1.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
#Deleting Rows where the Borough is not assigned
df2=df1
df2['Borough'] = df2['Borough'].str.strip()
df3 = df2
df3.drop(df3[df3.Borough == 'Not assigned'].index, inplace=True)

In [12]:
df3.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [13]:
#for Neighrbourhoods which are not assigned, assigning the value of the Borough
df3['Neighbourhood'] = df3['Neighbourhood'].str.strip()
df3.loc[df3['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df3['Borough']

In [14]:
df3.drop(df3.index[[211]], inplace=True)

Final Shape of the DataFrame after cleaning is (211, 3)

In [15]:
#Part 1 - Final Shape of the Required Data Frame
df3.shape

(211, 3)

In [16]:
df4 = df3.reset_index(drop=True)

In [17]:
df4.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#Part 2 - Merging with the Geo File

In [18]:
latlon = pd.read_csv('http://cocl.us/Geospatial_data')

In [19]:
#removing spaces
latlon['Postal Code'] = latlon['Postal Code'].str.lstrip()
df4['PostalCode'] = df4['PostalCode'].str.strip()

In [20]:
#creating the merged dataframe
merged_left = pd.merge(left=df4,right=latlon, left_on='PostalCode', right_on='Postal Code', how='left', indicator=True)

In [21]:
merged_left.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,_merge
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,both
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,both
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636,both
3,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636,both
4,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763,both


In [22]:
#Removing extra columns
merged_left.drop(["Postal Code", "_merge"], axis=1, inplace=True)

In [23]:
merged_left.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


Above is the required dataframe with the latitude and longitude added

# Part 3 - Generating Maps for the created dataframe